# 금연구역 주소 텍스트 전처리

동, 위도, 경도 3가지 위치정보를 확보하기 위해 kakao api를 활용하여 진행하고자 함.  
금연구역의 주소는 잡음이 많은 형태로 전처리 과정이 포함됨

In [1]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm

# import tensorflow as tf

import requests
import json


In [2]:
PATH = 'D:/project/데이터 분석 프로젝트/이노포스트/data'
# PATH = 'G:/다른 컴퓨터/My_desktop/project/데이터 분석 프로젝트/이노포스트/data'

df = pd.read_csv(PATH + '/03_금연구역/금연구역지정현황(원본).csv', encoding ='cp949')

In [3]:
df.head()

,구,구분,시설명,주소
0,중구,학교,굿모닝유치원,대전광역시 중구 어덕마을로 84-12 (중촌동)
1,중구,학교,금동유치원,대전광역시 중구 태평로152번길 63 (태평동)
2,중구,학교,꿈내리유치원,대전광역시 중구 당디로 45 (산성동)
3,중구,학교,남대전고등학교,대전광역시 중구 보문로111번길 63 (대사동)
4,중구,학교,대전고등학교,대전광역시 중구 대흥로 110 (대흥동)


## data check

In [4]:
df

,구,구분,시설명,주소
0,중구,학교,굿모닝유치원,대전광역시 중구 어덕마을로 84-12 (중촌동)
1,중구,학교,금동유치원,대전광역시 중구 태평로152번길 63 (태평동)
2,중구,학교,꿈내리유치원,대전광역시 중구 당디로 45 (산성동)
3,중구,학교,남대전고등학교,대전광역시 중구 보문로111번길 63 (대사동)
4,중구,학교,대전고등학교,대전광역시 중구 대흥로 110 (대흥동)
...,...,...,...,...
45602,유성구,복합용도,식당,대전광역시 유성구 학하동676-1
45603,유성구,복합용도,NaN,대전광역시 유성구 학하동713-12
45604,유성구,복합용도,NaN,대전광역시 유성구 학하동716-7
45605,유성구,복합용도,NaN,대전광역시 유성구 화암동157-6


In [5]:
df.isnull().sum()

구         0
구분        0
시설명    1737
주소        1
dtype: int64

In [6]:
df[df['주소'].isnull()] # 황금알식당은 폐업한 상점으로 제거

,구,구분,시설명,주소
3417,중구,"음식점(휴게, 일반, 제과)",황금알식당,NaN


In [7]:
df = df.dropna(subset=['주소'])
df.isnull().sum()

구         0
구분        0
시설명    1737
주소        0
dtype: int64

In [87]:
# 시설명이 없는 행 추출
df_null = df[df['시설명'].isnull()]
df_null

,구,구분,시설명,주소
12555,서구,연면적 1천제곱미터 이상 건축물,NaN,대전광역시 서구 복수동 258-0
12556,서구,연면적 1천제곱미터 이상 건축물,NaN,대전광역시 서구 복수동 271-1
12558,서구,연면적 1천제곱미터 이상 건축물,NaN,대전광역시 서구 복수동 274-13
12559,서구,연면적 1천제곱미터 이상 건축물,NaN,대전광역시 서구 복수동 275-18
12560,서구,연면적 1천제곱미터 이상 건축물,NaN,대전광역시 서구 복수동 276-0
...,...,...,...,...
45601,유성구,복합용도,NaN,대전광역시 유성구 하기동88-2
45603,유성구,복합용도,NaN,대전광역시 유성구 학하동713-12
45604,유성구,복합용도,NaN,대전광역시 유성구 학하동716-7
45605,유성구,복합용도,NaN,대전광역시 유성구 화암동157-6


시설명이 없는 건물을 삭제 후 진행했으나 추후 시설명 결측치를 제거하는게 옳은지 고민되어 이 부분을 다시 진행함

In [9]:
# df2 = df[~df['시설명'].isnull()]
# df2.reset_index(drop=True, inplace=True)
# df2.isnull().sum()

### 주소지

In [10]:
df.groupby('구분')['주소'].count().sort_values(ascending=False) 
# 금연구역에도 목욕장이 지정되어 있음, 목욕장 데이터와 중복

구분
음식점                  12556
일반음식점 및 휴게음식점         7718
음식점(휴게, 일반, 제과)       4247
학원                    2173
연면적 1천제곱미터 이상 건축물     2105
의료기관                  1906
담배판매업소 및 담배자동판매기      1367
복합건축물                 1206
어린이집                  1206
어린이놀이시설               1173
담배소매업소                1082
사회복지시설                 991
어린이운송용차량               771
체육시설                   727
담배소매인업소                702
신고체육시설                 680
어린이운송용 승합자동차           673
복합용도                   584
게임제공업소                 559
학교교과교습학원               437
학교                     424
공장                     339
사무용건축물                 277
실내체육시설                 251
청사                     234
체육시설업                  205
어린이운송용승합차              165
게임산업                   136
목욕장                    119
유치원                    104
식품자동판매기영업소              98
게임제공업소시설                94
어린이운송용 및 교통시설           82
도서관                     42
대규모점포                   32
지역아동센터                  24
공연장                     2

In [102]:
for idx, sentence in enumerate(df['주소']):
    if idx > 30: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

대전광역시 중구 어덕마을로 84-12 (중촌동)
대전광역시 중구 태평로152번길 63 (태평동)
대전광역시 중구 당디로 45 (산성동)
대전광역시 중구 보문로111번길 63 (대사동)
대전광역시 중구 대흥로 110 (대흥동)
대전광역시 중구 서문로 47 (문화동)
대전광역시 중구 계룡로918번길 103 (문화동)
대전광역시 중구 계룡로918번길 97 (문화동)
대전광역시 중구 계룡로918번길 97 (문화동)
대전광역시 중구 송리로 33 (문화동)
대전광역시 중구 송리로 11 (문화동)
대전광역시 중구 목동로 23 (목동)
대전광역시 중구 목동로 23 (목동)
대전광역시 중구 솔밭로16번길 17 (대사동)
대전광역시 중구 수도산로 12 (대흥동)
대전광역시 중구 문화로 200 (문화동)
대전광역시 중구 문화로 183 (문화동)
대전광역시 중구 문화로 183 (문화동)
대전광역시 중구 목중로19번길 70 (목동)
대전광역시 중구 목중로19번길 70 (목동)
대전광역시 중구 목동로 69 (목동)
대전광역시 중구 목동로 69 (목동)
대전광역시 중구 유등천동로292번길 89-14 (산성동)
대전광역시 중구 유등천동로292번길 89-14 (산성동)
대전광역시 중구 문창로 55 (부사동)
대전광역시 중구 문창로 55 (부사동)
대전광역시 중구 문화로218번길 63 (문화동)
대전광역시 중구 문화로78번안길 70 (문화동)
대전광역시 중구 문화로78번안길 70 (문화동)
대전광역시 중구 태평로 21 (태평동)
대전광역시 중구 태평로 21 (태평동)


### `()`와 `()안의 문자` 제거

In [11]:
df3 = df.copy()

In [12]:
# 번지주소 컬럼 생성
df3.insert(4, '번지주소', df3['주소'])

In [13]:
# ()와 (내용물) 제거
df3["주소"] = df3["주소"].str.replace(pat=r'\([^)]*\)', repl=r' ', regex=True)

In [14]:
# ()가 남아있는지 확인
df3[df3['주소'].str.contains('\)')]

,구,구분,시설명,주소,번지주소
260,중구,학교교과교습학원,목양해법영어학원,"대전광역시 중구 목동로 37 , 312호의 1/4, 313~316호 )","대전광역시 중구 목동로 37 , 312호의 1/4, 313~316호 (목동, 목양마..."
739,중구,어린이운송용 승합자동차,무궁화태권도장(태평관),대전광역시 중구 계백로1615번길 50 ),"대전광역시 중구 계백로1615번길 50 (유천동, 무궁화태권도장(태평관))"
877,중구,게임제공업소,쿵스,대전광역시 중구 중앙로156번길 16 앤비백화점),"대전광역시 중구 중앙로156번길 16 (은행동, (3층)앤비백화점)"
1226,중구,실내체육시설,스타복싱클럽,"대전광역시 중구 목중로 58, 3층중촌동)","대전광역시 중구 목중로 58, 3층중촌동)"
1947,중구,담배소매인업소,홈마트,대전광역시 중구 대둔산로 390 케이티 1층),대전광역시 중구 대둔산로 390 (산성동.(주)케이티 1층)
...,...,...,...,...,...
41819,유성구,학원,노은파워리딩학원,"대전광역시 유성구 노은로 144, 404호 405호 )","대전광역시 유성구 노은로 144, 404호 405호 (지족동,노은타운(씨네위))"
42026,유성구,학원,플레이팩토앤(N)창의교육학원,"대전광역시 유성구 지족로 374 , 301-2호, 302호 상가)","대전광역시 유성구 지족로 374 , 301-2호, 302호 (지족동, 옥타브(B동)상가)"
42027,유성구,학원,재테크경매학원,"대전광역시 유성구 지족로 374 , 410호,411호일부 상가)","대전광역시 유성구 지족로 374 , 410호,411호일부 (지족동,옥타브(B동)상가)"
42057,유성구,학원,워릭프랭클린어학원,"대전광역시 유성구 하기로 34-29 , 1층일부, 2층전체 )","대전광역시 유성구 하기로 34-29 , 1층일부, 2층전체 (하기동, 하기동 88-..."


In [15]:
# 남아있는 ) 제거
df3["주소"] = df3["주소"].str.replace(pat=r'\)', repl=r' ', regex=True)

In [16]:
df3[df3['주소'].str.contains('\)')]

,구,구분,시설명,주소,번지주소


In [17]:
# @ 제거
df3['주소'] = df3['주소'].str.replace('@', '아파트')
df3[df3['주소'].str.contains('@')]

,구,구분,시설명,주소,번지주소


### 중복된 공백 제거

In [18]:
print(df3['주소'].str.contains('  ').sum()) # 스페이스바 두번
print(df3['주소'].str.contains('   ').sum()) # 스페이스바 세번
print(df3['주소'].str.contains('    ').sum()) # 스페이스바 네번
print(df3['주소'].str.contains('      ').sum()) # 스페이스바 다섯번
print(df3['주소'].str.contains('       ').sum()) # 스페이스바 여섯번
print(df3['주소'].str.contains('.').sum())
print(df3['주소'].str.contains(',').sum())
print(df3['주소'].str.contains('외국').sum())
print(df3['주소'].str.contains('번지').sum()) # 번지 들어가면 번지주소?
print(df3['주소'].str.contains('동').sum()) # 동 들어가면 번지주소?

23492
2567
21
2
0
45606
12361
12
7950
30658


In [19]:
df3["주소"] = df3["주소"].str.replace(pat='  ', repl=r' ', regex=True)
print(df3['주소'].str.contains('  ').sum()) # 스페이스바 두번
print(df3['주소'].str.contains('   ').sum()) # 스페이스바 세번
print(df3['주소'].str.contains('    ').sum()) # 스페이스바 네번
print(df3['주소'].str.contains('     ').sum()) # 스페이스바 다섯번

2567
8
0
0


In [20]:
df3["주소"] = df3["주소"].str.replace(pat='  ', repl=r' ', regex=True)
print(df3['주소'].str.contains('  ').sum()) # 스페이스바 두번
print(df3['주소'].str.contains('   ').sum()) # 스페이스바 세번

8
0


In [21]:
df3["주소"] = df3["주소"].str.replace(pat='  ', repl=r' ', regex=True)
print(df3['주소'].str.contains('  ').sum()) # 스페이스바 두번
print(df3['주소'].str.contains('   ').sum()) # 스페이스바 세번

0
0


이후 중복된 공백은 여러번 실행으로 0이 될때까지 진행

In [22]:
df_space = df3
df_space[df_space['주소'].str.contains('  ')].count()

구       0
구분      0
시설명     0
주소      0
번지주소    0
dtype: int64

### 주소 형태 체크

In [23]:
df_space['주소'].nunique()

39270

In [24]:
df_space['주소']

0        대전광역시 중구 어덕마을로 84-12 
1        대전광역시 중구 태평로152번길 63 
2             대전광역시 중구 당디로 45 
3        대전광역시 중구 보문로111번길 63 
4            대전광역시 중구 대흥로 110 
                 ...          
45602       대전광역시 유성구 학하동676-1
45603      대전광역시 유성구 학하동713-12
45604       대전광역시 유성구 학하동716-7
45605       대전광역시 유성구 화암동157-6
45606       대전광역시 유성구 화암동63-10
Name: 주소, Length: 45606, dtype: object

### 시 분리

In [25]:
# 시작이 전부 대전광역시인지 확인
df_space['주소'].str[:5].unique()

array(['대전광역시', '대전 중구', '대전시 중', '대저너시 ', '대전시 충', '중구 서문', '대전 서구',
       '서울시 마', '보문산로3', '선화로43', '대전천서로', '부용로41', '당디로 9', '단재로 4',
       '유천로18', '충무로15', '대종로 2', '사득로 2', '천근로 3', '대종로 4', '대종로 1',
       '당골길 2', '테미로 5', '범골로 9', '성산로 2', '성산로14', '오류로 1', '충무로 1',
       '보문산로 ', '대종로55', '계백로15', '봉소루로 ', '돌다리로 ', '산서로 1', '계백로16',
       '대흥로 1', '문창로 2', '봉소루로5', '태평로14', '테미로 2', '계백로 1', '천근로 4',
       '대둔산로 ', '문화로10', '중앙로 2', '중구 오류', '중구 계백', '중구 계룡', '중구 중앙',
       '중구 대둔', '중구 태평', '중구 평촌', '중구 대종', ' ', '중구 방아', '중구 대전', '중구 동서',
       '중구 대흥', '수침로 9', '중앙로 1', '대전 대덕', '비공개시설', '\n대전광역', '대전시 대',
       '대전시 동', '외국 58', '외국 61', '외국 48', ' 대전광역', '대전시&n', '충청북도 ',
       '대전 동구', '대전 유성'], dtype=object)

* 대전광역시, 대전시, 대전 등 존재
* 제거하지 못한 특수기호들도 보임

#### `&nbsp;` 제거

In [26]:
# 대전시&n 가 뭐지?
df_space[df_space['주소'].str.contains('대전시&n')] 

,구,구분,시설명,주소,번지주소
14859,서구,학교,대전내동중학교,대전시&nbsp;서구&nbsp;배재로&nbsp;263,대전시&nbsp;서구&nbsp;배재로&nbsp;263 (변동)


In [27]:
# &nbsp; 17개 존재
df_space[df_space['주소'].str.contains('&nbsp;')].shape

(17, 5)

In [28]:
df_space["주소"] = df_space["주소"].str.replace(pat='&nbsp;', repl=r'', regex=True)

In [29]:
df_space[df_space['주소'].str.contains('&nbsp;')].shape # 제거완료

(0, 5)

#### 대전광역시 & 대전시 & 대전 제거

In [30]:
df_space["주소"] = df_space["주소"].str.replace(pat='대전광역시', repl=r'', regex=True)
df_space["주소"] = df_space["주소"].str.replace(pat='대전시', repl=r'', regex=True)
# df3["주소"] = df3["주소"].str.replace(pat='대전', repl=r'', regex=True) 
# 대전을 삭제하면 동대전로 대전로 등 같이 삭제됨

In [31]:
df_space['주소'].str[:4].unique()

array([' 중구 ', '대전 중', '대저너시', ' 충무로', '중구 서', ' 동구 ', '대전 서', ' 중구목',
       '서울시 ', ' 중구', '보문산로', '선화로4', '대전천서', '부용로4', '당디로 ', '단재로 ',
       '유천로1', '충무로1', '대종로 ', '사득로 ', '천근로 ', '당골길 ', '테미로 ', '범골로 ',
       '성산로 ', '성산로1', '오류로 ', '충무로 ', '대종로5', '계백로1', '봉소루로', '돌다리로',
       '산서로 ', '대흥로 ', '문창로 ', '태평로1', '계백로 ', '대둔산로', '문화로1', '중앙로 ',
       '중구 오', '중구 계', '중구 중', '중구 대', '중구 태', '중구 평', ' ', '중구 방',
       '중구 동', '수침로 ', ' 대덕구', '대전 대', '비공개시', '\n 대덕', ' 비래동', ' 송촌북',
       ' 벚꽃길', ' 계족로', ' 신탄진', ' 동춘당', ' 비래서', ' 오정로', ' 엣신탄', ' 덕암로',
       ' 석봉북', ' 계족산', ' 대전로', ' 한밭대', ' 동산초', ' 우암동', ' 대덕대', ' 중리북',
       ' 서구 ', '외국 5', '외국 6', '외국 4', '  서구', '서구가장', '서구도산', '서구배재',
       '서구원도', '서구벌곡', '서구둔산', ' 서구유', '서구신갈', '서구기성', '서구탄방', '서구동서',
       '서구정림', ' 서구월', '서구대덕', '서구관저', '서구복수', '충청북도', ' 서구조', ' 서구흑',
       '대전 동', '?동구?', ' 동구?', ' 동구옥', ' 동구백', ' 유성구', ' 전민로', '대전 유'],
      dtype=object)

* 대저너시, '  ', \n, ?, 서울시, 외국 등 확인
* 대전천서, 대전 서, 대전 중, 대전 대, 대전로, 대전 유, 대전 동 등 `대전` + `도로명` 형태 확인
    * 대전 뒤에 일괄적으로 공백 삽입 후 다시 형태 확인

#### 대저너시, '  ', \n, ?, 서울시 등 확인

In [32]:
# \n 제거
df_space[df_space['주소'].str.contains('\n')]

,구,구분,시설명,주소,번지주소
11914,대덕구,청사,수질연구소,\n 대덕구 송촌로 52,\n대전광역시 대덕구 송촌로 52
29172,동구,사회복지시설,참좋은집,"동구 성동로74, 102동 201호\n","대전광역시 동구 성동로74, 102동 201호\n(자양동, 동아아파트)"
29173,동구,사회복지시설,소망의집,"동구 계족로 409번길 33, 302호\n","대전광역시 동구 계족로 409번길 33, 302호\n(홍도동, 공원빌라)"
29174,동구,사회복지시설,신흥마을의집,"동구 옥천로 38, 101동 704호\n","대전광역시 동구 옥천로 38, 101동 704호\n(신흥동,신흥마을아파트)"
29175,동구,사회복지시설,샤인하우스,"동구 신기로 100, 607동104호 \n","대전광역시 동구 신기로 100, 607동104호 \n(가오동, 은어송마을아파트)"
29187,동구,사회복지시설,한빛지역아동센터,"동구 옥천로 180번길15,\n리라빌딩 301호","대전광역시 동구 옥천로 180번길15,\n리라빌딩 301호(판암1동)"
29198,동구,사회복지시설,파랑새휴먼지역아동센터,"동구 한밭대로1237번길\n45-14 ,2층","대전광역시 동구 한밭대로1237번길\n45-14(용전동),2층"
29267,동구,사회복지시설,주사랑 쉼터,"동구 대전로 542번길 121, 316/301\n","대전광역시 동구 대전로 542번길 121, 316/301\n(천동, 위드윌@)"
29268,동구,사회복지시설,대전밀알의집,동구 용운로 156번길 15\n,대전광역시 동구 용운로 156번길 15\n(용운동)
29271,동구,사회복지시설,해달별,"동구 계족로 489번길 73,\n101-101","대전광역시 동구 계족로 489번길 73,\n101-101(용전동, 한숲@)"


In [33]:
# \n 제거
df_space["주소"] = df_space["주소"].str.replace(pat='\n', repl=r' ', regex=True)

In [34]:
# ? 가 들어있는 행 확인
df_space[df_space['주소'].str.contains('\?')]

,구,구분,시설명,주소,번지주소
624,중구,어린이운송용 승합자동차,용인대온누리태권도장,"중구 태평동 422-7?; 버드내 상가 301호, 용인대온누리태권도장","대전광역시 중구 태평동 422-7?; 버드내 상가 301호, 용인대온누리태권도장"
1122,중구,실내체육시설,제일당구클럽,"중구 목중로 50, 402호 ?","대전광역시 중구 목중로 50, 402호(중촌동,제일프라자) ?"
16334,서구,학원,경희음악학원,서구 갈마로 276? 2층 세연음악학원,대전광역시 서구 갈마로 276? 2층 세연음악학원(내동)
16420,서구,학원,튼튼영어마스터클럽대전관저학원,서구 관저동로 66? 401호,대전광역시 서구 관저동로 66? 401호(관저동)
16487,서구,학원,관저디자인애니톡미술학원,서구 관저중로95번길 104? 4층,대전광역시 서구 관저중로95번길 104? 4층(관저동)
16681,서구,학원,둔산이화무용학원,서구 둔산남로9번길 43? 4층,대전광역시 서구 둔산남로9번길 43? 4층(둔산동)
16801,서구,학원,대치학원,서구 둔산로137번길 17? 4층일부,대전광역시 서구 둔산로137번길 17? 4층일부(둔산동)
16877,서구,학원,다빈치영재입시센터둔산학원,서구 둔산중로58번길 25? 2층전체,대전광역시 서구 둔산중로58번길 25? 2층전체(둔산동)
16878,서구,학원,메이몽어학원,서구 둔산중로58번길 25? 5층일부,대전광역시 서구 둔산중로58번길 25? 5층일부(둔산동)
16966,서구,학원,엠투수학학원,서구 문예로 65? 4층일부,대전광역시 서구 문예로 65? 4층일부(둔산동)


In [35]:
df_space["주소"] = df_space["주소"].str.replace(pat='\?', repl=r' ', regex=True) # ? 제거

In [36]:
df_space[df_space['주소'].str.contains('대저너시')]

,구,구분,시설명,주소,번지주소
422,중구,어린이운송용 승합자동차,오현유치원,"대저너시 중구 당디로6번길 69, 오현유치원, 오현어린이집, 오현태권도","대저너시 중구 당디로6번길 69, 오현유치원, 오현어린이집, 오현태권도"


In [37]:
# 대저너시 제거
df_space["주소"] = df_space["주소"].str.replace(pat='대저너시', repl=r' ', regex=True)

In [38]:
df_space[df_space['주소'].str.contains('서울시')]

,구,구분,시설명,주소,번지주소
785,중구,어린이운송용 승합자동차,오즈킨더유치원,"서울시 마포구 신촌로 170, 820호","서울시 마포구 신촌로 170, 820호"


In [39]:
# 서울시 행 삭제
df_space = df_space.drop([785], axis=0)

In [40]:
# 외국이 뭐지?
df_space[df_space['주소'].str.contains('외국')]

,구,구분,시설명,주소,번지주소
571,중구,어린이운송용 승합자동차,삼육외국어학원,"대전 중구 대사동 248-349번지 , 삼육외국어학원","대전 중구 대사동 248-349번지(1~3층), 삼육외국어학원"
773,중구,어린이운송용 승합자동차,삼육외국어학원,"중구 계룡로921번길 8, 1~3층 삼육외국어학원","대전광역시 중구 계룡로921번길 8, 1~3층(대사동) 삼육외국어학원"
777,중구,어린이운송용 승합자동차,삼육외국어학원,"중구 계룡로921번길 8, 1~3층 삼육외국어학원","대전광역시 중구 계룡로921번길 8, 1~3층(대사동) 삼육외국어학원"
811,중구,어린이운송용 승합자동차,삼육외국어학원,"중구 계룡로921번길 8,1~3층 삼육외국어학원","대전시 중구 계룡로921번길 8,1~3층(대사동) 삼육외국어학원"
819,중구,어린이운송용 승합자동차,삼육외국어학원,"중구 계룡로921번길 8, 1~3층 , 삼육외국어학원","대전시 중구 계룡로921번길 8, 1~3층(대사동), 삼육외국어학원"
13109,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 588-1,외국 588-1
13378,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 610-10,외국 610-10
13381,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 484-0,외국 484-0
13383,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 610-0,외국 610-0
13384,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 610-7,외국 610-7


In [41]:
# 외국으로 시작하는 지번 제거
df_space = df_space.drop([13109, 13378, 13381, 13383, 13384, 13385], axis=0) # 외국 삭제

In [42]:
df_space.reset_index(drop=True, inplace=True)
df_space.loc[13109]

구                        서구
구분        연면적 1천제곱미터 이상 건축물
시설명                     NaN
주소            서구 둔산동 1335-0
번지주소    대전광역시 서구 둔산동 1335-0
Name: 13109, dtype: object

In [43]:
# 다시 주소 형태 확인
df_space['주소'].str[:4].unique()

array([' 중구 ', '대전 중', '  중구', ' 충무로', '중구 서', ' 동구 ', '대전 서', ' 중구목',
       ' 중구', '보문산로', '선화로4', '대전천서', '부용로4', '당디로 ', '단재로 ', '유천로1',
       '충무로1', '대종로 ', '사득로 ', '천근로 ', '당골길 ', '테미로 ', '범골로 ', '성산로 ',
       '성산로1', '오류로 ', '충무로 ', '대종로5', '계백로1', '봉소루로', '돌다리로', '산서로 ',
       '대흥로 ', '문창로 ', '태평로1', '계백로 ', '대둔산로', '문화로1', '중앙로 ', '중구 오',
       '중구 계', '중구 중', '중구 대', '중구 태', '중구 평', ' ', '중구 방', '중구 동',
       '수침로 ', ' 대덕구', '대전 대', '비공개시', '  대덕', ' 비래동', ' 송촌북', ' 벚꽃길',
       ' 계족로', ' 신탄진', ' 동춘당', ' 비래서', ' 오정로', ' 엣신탄', ' 덕암로', ' 석봉북',
       ' 계족산', ' 대전로', ' 한밭대', ' 동산초', ' 우암동', ' 대덕대', ' 중리북', ' 서구 ',
       '  서구', '서구가장', '서구도산', '서구배재', '서구원도', '서구벌곡', '서구둔산', ' 서구유',
       '서구신갈', '서구기성', '서구탄방', '서구동서', '서구정림', ' 서구월', '서구대덕', '서구관저',
       '서구복수', '충청북도', ' 서구조', ' 서구흑', '대전 동', ' 동구옥', ' 동구백', ' 유성구',
       ' 전민로', '대전 유'], dtype=object)

* 앞자리가 공백으로 시작하는 것 확인 -> 앞 공백 제거
* 대전천서, 대전 서, 대전 중, 대전 대, 대전로, 대전 유, 대전 동 등 `대전` + `도로명` 형태 확인
    * 대전 뒤에 일괄적으로 공백 삽입 후 다시 형태 확인

In [44]:
# 대전 뒤에 공백 삽입
df_space['주소'] = df_space['주소'].str.replace('대전', '대전 ')
df_space[df_space['주소'].str.contains('대전')]

,구,구분,시설명,주소,번지주소
337,중구,어린이운송용 승합자동차,강남어린이집,대전 중구 계룡로873-11 강남어린이집,대전 중구 계룡로873-11 강남어린이집
338,중구,어린이운송용 승합자동차,해바라기어린이집,대전 중구 목동 목양마을아파트 단지내 해바라기어린이집,대전 중구 목동 목양마을@ 단지내 해바라기어린이집
339,중구,어린이운송용 승합자동차,오현유치원,대전 중구 문화동 780번지 오현유치원,대전 중구 문화동 780번지 오현유치원
342,중구,어린이운송용 승합자동차,꿈터어린이집,대전 중구 부사동 100번지 10호 꿈터어린이집,대전 중구 부사동 100번지 10호 꿈터어린이집
343,중구,어린이운송용 승합자동차,랄랄라어린이집,대전 중구 태평동 393번지 삼부아파트 410동 12호 랄랄라어린이집,대전 중구 태평동 393번지 삼부아파트 410동 12호 랄랄라어린이집
...,...,...,...,...,...
44247,유성구,어린이놀이시설,호산어린이공원,대전 유성구 학하동 730,대전 유성구 학하동 730
44248,유성구,어린이놀이시설,별빛어린이공원,대전 유성구 학하동 715-1,대전 유성구 학하동 715-1
44249,유성구,어린이놀이시설,학무정어린이공원,대전 유성구 학하동 763-4,대전 유성구 학하동 763-4
44250,유성구,어린이놀이시설,영당말어린이공원,대전 유성구 학하동 778-1,대전 유성구 학하동 778-1


In [45]:
df_space['주소'].str[:4].unique()

array([' 중구 ', '대전  ', '  중구', ' 충무로', '중구 서', ' 동구 ', ' 중구목', ' 중구',
       '보문산로', '선화로4', '대전 천', '부용로4', '당디로 ', '단재로 ', '유천로1', '충무로1',
       '대종로 ', '사득로 ', '천근로 ', '당골길 ', '테미로 ', '범골로 ', '성산로 ', '성산로1',
       '오류로 ', '충무로 ', '대종로5', '계백로1', '봉소루로', '돌다리로', '산서로 ', '대흥로 ',
       '문창로 ', '태평로1', '계백로 ', '대둔산로', '문화로1', '중앙로 ', '중구 오', '중구 계',
       '중구 중', '중구 대', '중구 태', '중구 평', ' ', '중구 방', '중구 동', '수침로 ',
       ' 대덕구', '비공개시', '  대덕', ' 비래동', ' 송촌북', ' 벚꽃길', ' 계족로', ' 신탄진',
       ' 동춘당', ' 비래서', ' 오정로', ' 엣신탄', ' 덕암로', ' 석봉북', ' 계족산', ' 대전 ',
       ' 한밭대', ' 동산초', ' 우암동', ' 대덕대', ' 중리북', ' 서구 ', '  서구', '서구가장',
       '서구도산', '서구배재', '서구원도', '서구벌곡', '서구둔산', ' 서구유', '서구신갈', '서구기성',
       '서구탄방', '서구동서', '서구정림', ' 서구월', '서구대덕', '서구관저', '서구복수', '충청북도',
       ' 서구조', ' 서구흑', ' 동구옥', ' 동구백', ' 유성구', ' 전민로'], dtype=object)

#### 다시 중복된 공백 제거

In [46]:
# 중복 공백 확인
print(df_space['주소'].str.contains('  ').sum()) # 스페이스바 두번
print(df_space['주소'].str.contains('   ').sum()) # 스페이스바 세번
print(df_space['주소'].str.contains('    ').sum()) # 스페이스바 네번

701
2
0


In [47]:
# 전부 0이 될때까지 여러번 실행
df_space["주소"] = df_space["주소"].str.replace(pat='  ', repl=r' ', regex=True)
print(df_space['주소'].str.contains('  ').sum()) # 스페이스바 두번
print(df_space['주소'].str.contains('   ').sum()) # 스페이스바 세번

2
0


In [48]:
df_space['주소'].str[:4].unique()

array([' 중구 ', '대전 중', ' 충무로', '중구 서', ' 동구 ', '대전 서', ' 중구목', ' 중구',
       '보문산로', '선화로4', '대전 천', '부용로4', '당디로 ', '단재로 ', '유천로1', '충무로1',
       '대종로 ', '사득로 ', '천근로 ', '당골길 ', '테미로 ', '범골로 ', '성산로 ', '성산로1',
       '오류로 ', '충무로 ', '대종로5', '계백로1', '봉소루로', '돌다리로', '산서로 ', '대흥로 ',
       '문창로 ', '태평로1', '계백로 ', '대둔산로', '문화로1', '중앙로 ', '중구 오', '중구 계',
       '중구 중', '중구 대', '중구 태', '중구 평', ' ', '중구 방', '중구 동', '수침로 ',
       ' 대덕구', '대전 대', '비공개시', ' 비래동', ' 송촌북', ' 벚꽃길', ' 계족로', ' 신탄진',
       ' 동춘당', ' 비래서', ' 오정로', ' 엣신탄', ' 덕암로', ' 석봉북', ' 계족산', ' 대전 ',
       ' 한밭대', ' 동산초', ' 우암동', ' 대덕대', ' 중리북', ' 서구 ', '서구가장', '서구도산',
       '서구배재', '서구원도', '서구벌곡', '서구둔산', ' 서구유', '서구신갈', '서구기성', '서구탄방',
       '서구동서', '서구정림', ' 서구월', '서구대덕', '서구관저', '서구복수', '충청북도', ' 서구조',
       ' 서구흑', '대전 동', ' 동구옥', ' 동구백', ' 유성구', ' 전민로', '대전 유'],
      dtype=object)

#### 맨 앞 공백제거

In [49]:
df_space["주소"] = df_space["주소"].str.lstrip()

In [50]:
df_space['주소'].str[:3].unique()

array(['중구 ', '대전 ', '충무로', '동구 ', '중구목', '중구', '보문산', '선화로', '부용로',
       '당디로', '단재로', '유천로', '대종로', '사득로', '천근로', '당골길', '테미로', '범골로',
       '성산로', '오류로', '계백로', '봉소루', '돌다리', '산서로', '대흥로', '문창로', '태평로',
       '대둔산', '문화로', '중앙로', '', '수침로', '대덕구', '비공개', '비래동', '송촌북', '벚꽃길',
       '계족로', '신탄진', '동춘당', '비래서', '오정로', '엣신탄', '덕암로', '석봉북', '계족산',
       '한밭대', '동산초', '우암동', '대덕대', '중리북', '서구 ', '서구가', '서구도', '서구배',
       '서구원', '서구벌', '서구둔', '서구유', '서구신', '서구기', '서구탄', '서구동', '서구정',
       '서구월', '서구대', '서구관', '서구복', '충청북', '서구조', '서구흑', '동구옥', '동구백',
       '유성구', '전민로'], dtype=object)

In [51]:
df_space

,구,구분,시설명,주소,번지주소
0,중구,학교,굿모닝유치원,중구 어덕마을로 84-12,대전광역시 중구 어덕마을로 84-12 (중촌동)
1,중구,학교,금동유치원,중구 태평로152번길 63,대전광역시 중구 태평로152번길 63 (태평동)
2,중구,학교,꿈내리유치원,중구 당디로 45,대전광역시 중구 당디로 45 (산성동)
3,중구,학교,남대전고등학교,중구 보문로111번길 63,대전광역시 중구 보문로111번길 63 (대사동)
4,중구,학교,대전고등학교,중구 대흥로 110,대전광역시 중구 대흥로 110 (대흥동)
...,...,...,...,...,...
45594,유성구,복합용도,식당,유성구 학하동676-1,대전광역시 유성구 학하동676-1
45595,유성구,복합용도,NaN,유성구 학하동713-12,대전광역시 유성구 학하동713-12
45596,유성구,복합용도,NaN,유성구 학하동716-7,대전광역시 유성구 학하동716-7
45597,유성구,복합용도,NaN,유성구 화암동157-6,대전광역시 유성구 화암동157-6


## 구 분리

대전시는 유성구, 대덕구, 중구, 서구, 동구 다섯가지 지역구로 구성됨  
주소에서 구를 분리하는 작업
 * 구 뒤에 공백 삽업


In [52]:
df_gu = df_space.copy()

In [53]:
df_gu['구'].unique()

array(['중구', '대덕구', '서구', '동구', '유성구'], dtype=object)

#### 구 뒤에 공백 삽입

In [54]:
# 구 뒤에 공백 삽입
df_gu['주소'] = df_gu['주소'].str.replace('중구', '중구 ')
df_gu['주소'] = df_gu['주소'].str.replace('대덕구', '대덕구 ')
df_gu['주소'] = df_gu['주소'].str.replace('서구', '서구 ')
df_gu['주소'] = df_gu['주소'].str.replace('동구', '동구 ')
df_gu['주소'] = df_gu['주소'].str.replace('유성구', '유성구 ')

#### `**구` 제거

In [55]:
df_gu["주소"] = df_gu["주소"].str.replace(pat='중구 ', repl=r'', regex=True)
df_gu["주소"] = df_gu["주소"].str.replace(pat='대덕구 ', repl=r'', regex=True)
df_gu["주소"] = df_gu["주소"].str.replace(pat='서구 ', repl=r'', regex=True)
df_gu["주소"] = df_gu["주소"].str.replace(pat='동구 ', repl=r'', regex=True)
df_gu["주소"] = df_gu["주소"].str.replace(pat='유성구 ', repl=r'', regex=True)

In [56]:
df_gu[df_gu['주소'].str.contains('중구|대덕구|서구|동구|유성구')]

,구,구분,시설명,주소,번지주소


In [57]:
# 공백제거
df_gu["주소"] = df_gu["주소"].str.replace(pat='  ', repl=r' ', regex=True)
df_gu["주소"] = df_gu["주소"].str.replace(pat='  ', repl=r' ', regex=True)

In [58]:
print(df_gu['주소'].str.contains('  ').sum()) # 스페이스바 두번
print(df_gu['주소'].str.contains('   ').sum()) # 스페이스바 세번
print(df_gu['주소'].str.contains('    ').sum()) # 스페이스바 네번

0
0
0


In [59]:
# 맨 앞에 공백 제거
df_gu["주소"] = df_gu["주소"].str.lstrip()

In [60]:
df_gu['주소'].str[:2].unique()

array(['어덕', '태평', '당디', '보문', '대흥', '서문', '계룡', '송리', '목동', '솔밭', '수도',
       '문화', '목중', '유등', '문창', '대종', '대둔', '선화', '부사', '오류', '모암', '계백',
       '수침', '유천', '사득', '산성', '남달', '학고', '동서', '평촌', '천근', '중앙', '충무',
       '중교', '과례', '대전', '사정', '중촌', '석교', '돌다', '용두', '산서', '문로', '테미',
       '동부', '호동', '', '봉소', '안영', '뿌리', '인창', '목척', '부용', '단재', '당골',
       '범골', '성산', '신기', '대사', '침산', '방아', '우암', '운남', '정생', '잔디', '가재',
       '한남', '신탄', '홍도', '계족', '중리', '덕암', '동춘', '대덕', '비래', '한밭', '석봉',
       '옛신', '송촌', '오정', '신일', '대청', '아리', '선비', '대화', '문평', '동심', '벚꽃',
       '대오', '생산', '신상', '쌍청', '목상', '법동', '갑천', '상서', '늘봄', '장동', '산디',
       '남경', '새말', '청자', '태양', '산업', '회덕', '방두', '동성', '비공', '미호', '와동',
       '신대', '읍내', '연축', '삼정', '용호', '동산', '청림', '무지', '엣신', '흥룡', '다사',
       '복수', '용문', '둔산', '도마', '탄방', '변동', '관저', '괴정', '월평', '도안', '갈마',
       '만년', '가수', '정림', '내동', '가장', '오동', '흑석', '장안', '우명', '매노', '산직',
       '괴곡', '원정', '원도', '청사', '구봉', '신갈', '배재', '중반'

In [61]:
df_gu

,구,구분,시설명,주소,번지주소
0,중구,학교,굿모닝유치원,어덕마을로 84-12,대전광역시 중구 어덕마을로 84-12 (중촌동)
1,중구,학교,금동유치원,태평로152번길 63,대전광역시 중구 태평로152번길 63 (태평동)
2,중구,학교,꿈내리유치원,당디로 45,대전광역시 중구 당디로 45 (산성동)
3,중구,학교,남대전고등학교,보문로111번길 63,대전광역시 중구 보문로111번길 63 (대사동)
4,중구,학교,대전고등학교,대흥로 110,대전광역시 중구 대흥로 110 (대흥동)
...,...,...,...,...,...
45594,유성구,복합용도,식당,학하동676-1,대전광역시 유성구 학하동676-1
45595,유성구,복합용도,NaN,학하동713-12,대전광역시 유성구 학하동713-12
45596,유성구,복합용도,NaN,학하동716-7,대전광역시 유성구 학하동716-7
45597,유성구,복합용도,NaN,화암동157-6,대전광역시 유성구 화암동157-6


## 도로명주소 전처리 시도

도로명주소는 

`행정구역` `도로명건물번호` `,` `상세주소` 로 구분됨.  
첫 `,`를 기준으로 나누면 도로명주소의 훼손없이 분리 가능

In [62]:
df_split = df_gu.copy()
df_split['세부주소'] = np.nan
df_split

,구,구분,시설명,주소,번지주소,세부주소
0,중구,학교,굿모닝유치원,어덕마을로 84-12,대전광역시 중구 어덕마을로 84-12 (중촌동),NaN
1,중구,학교,금동유치원,태평로152번길 63,대전광역시 중구 태평로152번길 63 (태평동),NaN
2,중구,학교,꿈내리유치원,당디로 45,대전광역시 중구 당디로 45 (산성동),NaN
3,중구,학교,남대전고등학교,보문로111번길 63,대전광역시 중구 보문로111번길 63 (대사동),NaN
4,중구,학교,대전고등학교,대흥로 110,대전광역시 중구 대흥로 110 (대흥동),NaN
...,...,...,...,...,...,...
45594,유성구,복합용도,식당,학하동676-1,대전광역시 유성구 학하동676-1,NaN
45595,유성구,복합용도,NaN,학하동713-12,대전광역시 유성구 학하동713-12,NaN
45596,유성구,복합용도,NaN,학하동716-7,대전광역시 유성구 학하동716-7,NaN
45597,유성구,복합용도,NaN,화암동157-6,대전광역시 유성구 화암동157-6,NaN


### 콤마가 있는 값 분리

In [63]:
df_split_com = df_split[df_split['주소'].str.contains(',')]
df_split_ncom = df_split[~df_split['주소'].str.contains(',')]
print(df_split_com.shape)
print(df_split_ncom.shape)

(12360, 6)
(33239, 6)


In [64]:
df_split_com 

,구,구분,시설명,주소,번지주소,세부주소
95,중구,학교교과교습학원,중앙독서실,"목중로26번길 , 83","대전광역시 중구 목중로26번길 , 83 (중촌동, 현대아파트)",NaN
96,중구,학교교과교습학원,예녹피아노음악종합학원,"당디로 45 , 201호,203호,204호","대전광역시 중구 당디로 45 , 201호,203호,204호 (산성동, 우성아파트)",NaN
97,중구,학교교과교습학원,단비종합학원,"보문로 180 1층,2층,3층","대전광역시 중구 보문로 180 1층,2층,3층 (대흥동)",NaN
98,중구,학교교과교습학원,국제학원,"대둔산로 437 , 3층","대전광역시 중구 대둔산로 437 , 3층 (산성동)",NaN
100,중구,학교교과교습학원,두루음악학원,"당디로6번길 76 , 403호","대전광역시 중구 당디로6번길 76 , 403호 (문화동)",NaN
...,...,...,...,...,...,...
44528,유성구,지역아동센터,호산나반석지역아동센터,"지족로 218-33, 2층","대전광역시 유성구 지족로 218-33, 2층(지족동)",NaN
44529,유성구,지역아동센터,학교밖 청소년지원센터 꿈드림,"유성대로 798-1, 2층","대전광역시 유성구 유성대로 798-1, 2층(장대동)",NaN
44531,유성구,만화대여업소,놀숲 충남대점,"한밭대로 400, 3층","대전광역시 유성구 한밭대로 400, 3층 (궁동)",NaN
44542,유성구,숙박업소,호텔스카이파크 대전1호점,"테크노중앙로 161, 호텔 스카이파크","대전광역시 유성구 테크노중앙로 161, 호텔 스카이파크 (용산동)",NaN


In [65]:
# 첫번째 콤마를 기준으로 분리
a = df_split_com['주소'].str.split(',', expand=True, n=1)

In [66]:
a

,0,1
95,목중로26번길,83
96,당디로 45,"201호,203호,204호"
97,보문로 180 1층,"2층,3층"
98,대둔산로 437,3층
100,당디로6번길 76,403호
...,...,...
44528,지족로 218-33,2층
44529,유성대로 798-1,2층
44531,한밭대로 400,3층
44542,테크노중앙로 161,호텔 스카이파크


In [67]:
# df_split_com에 a 값 대체
df_split_com['주소'] = a[0].copy()
df_split_com['세부주소'] = a[1].copy()

c:\Users\JBY\anaconda3\envs\3.7\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\JBY\anaconda3\envs\3.7\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [68]:
df_split_com

,구,구분,시설명,주소,번지주소,세부주소
95,중구,학교교과교습학원,중앙독서실,목중로26번길,"대전광역시 중구 목중로26번길 , 83 (중촌동, 현대아파트)",83
96,중구,학교교과교습학원,예녹피아노음악종합학원,당디로 45,"대전광역시 중구 당디로 45 , 201호,203호,204호 (산성동, 우성아파트)","201호,203호,204호"
97,중구,학교교과교습학원,단비종합학원,보문로 180 1층,"대전광역시 중구 보문로 180 1층,2층,3층 (대흥동)","2층,3층"
98,중구,학교교과교습학원,국제학원,대둔산로 437,"대전광역시 중구 대둔산로 437 , 3층 (산성동)",3층
100,중구,학교교과교습학원,두루음악학원,당디로6번길 76,"대전광역시 중구 당디로6번길 76 , 403호 (문화동)",403호
...,...,...,...,...,...,...
44528,유성구,지역아동센터,호산나반석지역아동센터,지족로 218-33,"대전광역시 유성구 지족로 218-33, 2층(지족동)",2층
44529,유성구,지역아동센터,학교밖 청소년지원센터 꿈드림,유성대로 798-1,"대전광역시 유성구 유성대로 798-1, 2층(장대동)",2층
44531,유성구,만화대여업소,놀숲 충남대점,한밭대로 400,"대전광역시 유성구 한밭대로 400, 3층 (궁동)",3층
44542,유성구,숙박업소,호텔스카이파크 대전1호점,테크노중앙로 161,"대전광역시 유성구 테크노중앙로 161, 호텔 스카이파크 (용산동)",호텔 스카이파크


In [69]:
# 병합
df_road = pd.concat([df_split_com, df_split_ncom], 
                    axis=0,
                    join='outer'
                    )

df_road.reset_index(drop=True)

,구,구분,시설명,주소,번지주소,세부주소
0,중구,학교교과교습학원,중앙독서실,목중로26번길,"대전광역시 중구 목중로26번길 , 83 (중촌동, 현대아파트)",83
1,중구,학교교과교습학원,예녹피아노음악종합학원,당디로 45,"대전광역시 중구 당디로 45 , 201호,203호,204호 (산성동, 우성아파트)","201호,203호,204호"
2,중구,학교교과교습학원,단비종합학원,보문로 180 1층,"대전광역시 중구 보문로 180 1층,2층,3층 (대흥동)","2층,3층"
3,중구,학교교과교습학원,국제학원,대둔산로 437,"대전광역시 중구 대둔산로 437 , 3층 (산성동)",3층
4,중구,학교교과교습학원,두루음악학원,당디로6번길 76,"대전광역시 중구 당디로6번길 76 , 403호 (문화동)",403호
...,...,...,...,...,...,...
45594,유성구,복합용도,식당,학하동676-1,대전광역시 유성구 학하동676-1,NaN
45595,유성구,복합용도,NaN,학하동713-12,대전광역시 유성구 학하동713-12,NaN
45596,유성구,복합용도,NaN,학하동716-7,대전광역시 유성구 학하동716-7,NaN
45597,유성구,복합용도,NaN,화암동157-6,대전광역시 유성구 화암동157-6,NaN


## Kakao api

### kakao api 검색을 위한 컬럼 생성 (구 + 주소)

In [70]:
df_kakao = df_road.copy()
df_kakao['kakao 주소'] = np.nan
df_kakao

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소
95,중구,학교교과교습학원,중앙독서실,목중로26번길,"대전광역시 중구 목중로26번길 , 83 (중촌동, 현대아파트)",83,NaN
96,중구,학교교과교습학원,예녹피아노음악종합학원,당디로 45,"대전광역시 중구 당디로 45 , 201호,203호,204호 (산성동, 우성아파트)","201호,203호,204호",NaN
97,중구,학교교과교습학원,단비종합학원,보문로 180 1층,"대전광역시 중구 보문로 180 1층,2층,3층 (대흥동)","2층,3층",NaN
98,중구,학교교과교습학원,국제학원,대둔산로 437,"대전광역시 중구 대둔산로 437 , 3층 (산성동)",3층,NaN
100,중구,학교교과교습학원,두루음악학원,당디로6번길 76,"대전광역시 중구 당디로6번길 76 , 403호 (문화동)",403호,NaN
...,...,...,...,...,...,...,...
45594,유성구,복합용도,식당,학하동676-1,대전광역시 유성구 학하동676-1,NaN,NaN
45595,유성구,복합용도,NaN,학하동713-12,대전광역시 유성구 학하동713-12,NaN,NaN
45596,유성구,복합용도,NaN,학하동716-7,대전광역시 유성구 학하동716-7,NaN,NaN
45597,유성구,복합용도,NaN,화암동157-6,대전광역시 유성구 화암동157-6,NaN,NaN


In [71]:
df_kakao['kakao 주소'] = df_kakao['구'].str.cat(df_kakao['주소'], sep=' ')

In [72]:
df_kakao.reset_index(drop=True, inplace=True)
df_kakao.head(2)

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소
0,중구,학교교과교습학원,중앙독서실,목중로26번길,"대전광역시 중구 목중로26번길 , 83 (중촌동, 현대아파트)",83,중구 목중로26번길
1,중구,학교교과교습학원,예녹피아노음악종합학원,당디로 45,"대전광역시 중구 당디로 45 , 201호,203호,204호 (산성동, 우성아파트)","201호,203호,204호",중구 당디로 45


In [73]:
df_kakao2 = df_kakao

df_kakao2['동'] = np.nan 
df_kakao2['위도'] = np.nan
df_kakao2['경도'] = np.nan

df_kakao2.shape

(45599, 10)

## 1차 전저리 데이터프레임 저장

In [74]:
df_kakao2.to_csv(PATH + "/03_금연구역/금연구역_장병용_1차_텍스트전처리.csv", index=False, encoding='cp949')

### kakao api 세팅

In [75]:
# rest키 발급 https://developers.kakao.com/console/app

# 병용 474aa3bdba9bd34f1560a7812fafeb9b
# 재인 08d64ac0652c8f9e1c4ea45118910c53
# 수영 616734aa8214dbf958cfaebd6bd41aec
# 태희 04e7e5d98a2c026956de6095963f28f8

rest_api_key = "474aa3bdba9bd34f1560a7812fafeb9b"

In [76]:
class KakaoLocalAPI:
    """
    Kakao Local API Controller
    """
    def __init__(self, rest_api_key):
        """
        Rest API key 초기화 및 기능 별 url 설정
        """
        
        # REST API key 설정
        self.rest_api_key = rest_api_key
        self.headers = {"Authorization": "KakaoAK {}".format(rest_api_key)}
        
        # 01 주소검색
        self.URL_01 = "https://dapi.kakao.com/v2/local/search/address.json"
        
    def search_address(self, query, analyze_type = None, page = None, size = None):
        """
        01 주소 검색
        """
        params = {"query": f"{query}"}
    
        if analyze_type != None:
            params["analyze_type"]=f"{analyze_type}"
    
        elif page != None:
            params['page'] = f"{page}"
    
        elif size != None:
            params['size'] = f"{size}"
    
        res = requests.get(self.URL_01, headers = self.headers, params = params)
        document = json.loads(res.text)
    
        return document

In [77]:
kakao = KakaoLocalAPI(rest_api_key)

In [78]:
add_list = df_kakao['kakao 주소']
add_list[0]

'중구 목중로26번길 '

In [79]:
# api 호출 확인
kakao.search_address(add_list[0]) 

{'documents': [{'address': None,
   'address_name': '대전 중구 목중로26번길',
   'address_type': 'ROAD',
   'road_address': {'address_name': '대전 중구 목중로26번길',
    'building_name': '',
    'main_building_no': '',
    'region_1depth_name': '대전',
    'region_2depth_name': '중구',
    'region_3depth_name': '중촌동',
    'road_name': '목중로26번길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.412961781981',
    'y': '36.3375130575055',
    'zone_no': ''},
   'x': '127.412961781981',
   'y': '36.3375130575055'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

### 동, 위경도 추출

In [80]:
from tqdm import tqdm
from tqdm import trange

In [81]:
# 1시간 42분 걸림
for i in tqdm(range(len(add_list))):
    try :
        df_kakao2['동'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['address']['region_3depth_name']
        df_kakao2['경도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['x']
        df_kakao2['위도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['y']
    except:
        pass

print(df_kakao2)

  0%|          | 0/45599 [00:00<?, ?it/s]C:\Users\JBY\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|██████████| 45599/45599 [1:36:02<00:00,  7.91it/s]  

         구        구분          시설명          주소  \
0       중구  학교교과교습학원        중앙독서실    목중로26번길    
1       중구  학교교과교습학원  예녹피아노음악종합학원     당디로 45    
2       중구  학교교과교습학원       단비종합학원  보문로 180 1층   
3       중구  학교교과교습학원         국제학원   대둔산로 437    
4       중구  학교교과교습학원       두루음악학원  당디로6번길 76    
...    ...       ...          ...         ...   
45594  유성구      복합용도           식당    학하동676-1   
45595  유성구      복합용도          NaN   학하동713-12   
45596  유성구      복합용도          NaN    학하동716-7   
45597  유성구      복합용도          NaN    화암동157-6   
45598  유성구      복합용도          NaN    화암동63-10   

                                                번지주소              세부주소  \
0                대전광역시 중구 목중로26번길  , 83 (중촌동, 현대아파트)               83    
1      대전광역시 중구 당디로 45 , 201호,203호,204호 (산성동, 우성아파트)   201호,203호,204호    
2                    대전광역시 중구 보문로 180 1층,2층,3층 (대흥동)            2층,3층    
3                       대전광역시 중구 대둔산로 437 , 3층 (산성동)               3층    
4                    대전광역시 중구 당디로6번길 76 ,

#### kakao api 

45598건 중 12833건 실패

In [82]:
df_kakao2.isnull().sum()

구               0
구분              0
시설명          1732
주소              0
번지주소            0
세부주소        33239
kakao 주소        0
동           12833
위도          12833
경도          12833
dtype: int64

## 성공한 데이터프레임과 실패한 데이터프레임 저장

In [83]:
df_fail = df_kakao2[df_kakao2['위도'].isnull()]
df_save = df_kakao2[~df_kakao2['위도'].isnull()]

print(df_fail.shape)
print(df_save.shape)

(12833, 10)
(32766, 10)


In [106]:
df_save.to_csv(PATH + "/03_금연구역/금연구역_장병용_전처리_성공(32766).csv", index=False, encoding='cp949')
df_fail.to_csv(PATH + "/03_금연구역/금연구역_장병용_전처리_실패(12833).csv", index=False, encoding='cp949')